# Plotting phase space



In [1]:
# some preliminary settings and functions

import sys
sys.path.append("/home/aunai/build_phare")
sys.path.append("/home/aunai/PHARE/pyphare/")
import os
import numpy as np
import pyphare
from pyphare.pharesee.hierarchy import finest_data, finest_part_data
from pyphare.pharesee.hierarchy import hierarchy_from
from pyphare.pharesee.plotting import zoom_effect
import pyphare.core.box as boxm
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
%matplotlib widget

## Run001

In [2]:
def finest_part_data(hierarchy):
    """
    returns a dict {popname : Particles}
    Particles contained in the dict are those from
    the finest patches available at a given location
    """
    from .particles import remove
    from copy import deepcopy
    
    # we are going to return a dict {popname : Particles}
    # we prepare it with population names
    aPatch = hierarchy.patch_levels[0].patches[0]
    particles = {popname:{} for popname in aPatch.patch_datas.keys()}

    # our strategy is to explore the hierarchy from the finest
    # level to the coarsest. at Each level we keep only particles
    # that are in cells that are not overlaped by finer boxes
    
    # this dict keeps boxes for patches at each level
    # each level will thus need this dict to see next finer boxes
    lvlPatchBoxes = {ilvl:[] for ilvl in range(hierarchy.finest_level()+1)}
    

    for ilvl in range(hierarchy.finest_level()+1)[::-1]:
        plvl = hierarchy.patch_levels[ilvl]
        for ip, patch in enumerate(plvl.patches):
            lvlPatchBoxes[ilvl].append(patch.box)
            for popname, pdata in patch.patch_datas.items():
                
                # if we're at the finest level
                # we need to keep all particles
                if ilvl == hierarchy.finest_level():        
                    if ilvl not in particles[popname]:
                        particles[popname][ilvl] = pdata.dataset
                    else:
                        particles[popname][ilvl].add(pdata.dataset)
                                            
                # if there is a finer level
                # we need to keep only those of the current patch
                # that are not in cells overlaped by finer patch boxes
                else:
                    icells = pdata.dataset.iCells
                    parts = pdata.dataset
                    create = True
                    for finerBox in lvlPatchBoxes[ilvl+1]:
                        coarseFinerBox = boxm.coarsen(finerBox, 2)
                        within = np.where((icells >= coarseFinerBox.lower[0]) &\
                                                 (icells <= coarseFinerBox.upper[0]))[0]                        
                        if create:
                            toRemove = within
                            create=False
                        else:
                            toRemove = np.concatenate((toRemove, within))
                            
                    parts = remove(parts, toRemove)
                    if parts is not None:
                        if ilvl not in particles[popname]:
                            particles[popname][ilvl] = parts
                        else:
                            particles[popname][ilvl].add(parts)
                                        
    return particles

In [101]:
def dist_plot(particles, **kwargs):
    """
    plot the phase space of given particles
    particles can be of type Particles, list(Particles), dict{popname:Particles}
    
    kwargs:
    * axis : ("x", "Vx"), ("x", "Vy"), ("x", "Vz"), ("Vx", "Vy") (default) --
       ("Vx", "Vz"), ("Vy", "Vz")
    * bins :  number of bins in each dimension, default is (50,50)
    * sigma : width of the gaussian filter, default is (0,0)
    * norm  : histogram will be normed to Normalize(0,norm)
    * kde : (default False) : adds contours of kernel density estimate
    * title : (str) title of the plot
    * xlabel, ylabel
    * xlim, ylim
    * bulk : (bool) (default : False), adds vertical/horizontal lines -- 
             at in-plane bulk velocity for velocity axis
    * filename : (str) if exists, save plot to figure under that name
    
    return value : fig,ax
    """
    from pyphare.pharesee.particles import Particles, aggregate
    from matplotlib.colors import Normalize
    
    if isinstance(particles, list):
        particles = aggregate(particles)
    elif isinstance(particles, dict):
        particles = aggregate([p for p in particles.values()])
    
    if not isinstance(particles, Particles):
        raise ValueError("Error, 'particles' type should be Particles, list or dict")
    
    from scipy.ndimage import gaussian_filter as gf
    if "ax" not in kwargs:
        fig, ax = plt.subplots()
    else:
        ax = kwargs["ax"]
        fig = ax.figure
    axis = kwargs.get("axis",("Vx","Vy"))
    vaxis = {"Vx":0, "Vy":1, "Vz":2}
    
    if axis[0] in vaxis:
        x = particles.v[:,vaxis[axis[0]]]
    elif axis[0] == "x":
        x = particles.x
    if axis[1] in vaxis:
        y = particles.v[:,vaxis[axis[1]]]

    bins = kwargs.get("bins", (50,50))
    h, xh, yh  = np.histogram2d(x, y,
              bins=kwargs.get("bins", bins),
              weights=particles.weights)

    xc = 0.5*(xh[1:] + xh[:-1])
    yc = 0.5*(yh[1:] + yh[:-1])
    sig = kwargs.get("sigma", (0,0))
    hm = kwargs.get("norm", h.max())
    ax.pcolormesh(xh, yh, gf(h.T, sigma=sig), cmap="jet", norm=Normalize(0,hm))

    if kwargs.get("kde",False) is True:
        sns.kdeplot(x, y, ax=ax, color="w")


    ax.set_title(kwargs.get("title",""))
    ax.set_xlabel(kwargs.get("xlabel", axis[0]))
    ax.set_ylabel(kwargs.get("ylabel", axis[1]))
    if "xlim" in kwargs:
        ax.set_xlim(kwargs["xlim"])
    if "ylim" in kwargs:
        ax.set_ylim(kwargs["ylim"])
    ax.legend()

    if "bulk" in kwargs:
        if kwargs["bulk"] is True:
            if axis[0] in vaxis:
                ax.axvline(particles.v[:,vaxis[axis[0]]].mean(), color="w",ls="--")
            if axis[1] in vaxis:
                ax.axhline(particles.v[:,vaxis[axis[1]]].mean(), color="w",ls="--")


    if "filename" in kwargs:
        fig.savefig(kwargs["filename"])
    
    return fig,ax
        
        
def hdist_plot(self, **kwargs):    
    """
    plot phase space of a particle hierarchy
    """
    import copy
    usr_lvls = kwargs.get("levels",(0,))
    finest = kwargs.get("finest", False)
    pops = kwargs.get("pop",[])
    time = kwargs.get("time", self.times()[0])
    axis = kwargs.get("axis", ("Vx", "Vy"))
    if finest:
        final = finest_part_data(self)
        vmin = kwargs.get("vmin", -2)
        vmax = kwargs.get("vmax", 2)
        dv = kwargs.get("dv", 0.05)
        vbins = vmin + dv*np.arange(int((vmax-vmin)/dv))
        
        for ilvl, lvl in self.levels(time).items():
            
            if axis[0] == "x":
                dx = lvl.patches[0].layout.dl[0]
                xmin = min([ p.layout.box.lower[0]*dx for p in lvl.patches])
                xmax = max([ (p.layout.box.upper[0]+1)*dx for p in lvl.patches])
                nx = int((xmax-xmin)/dx) +1
                xbins = xmin + dx*np.arange(nx)
                if ilvl==0:
                    fig, ax = dist_plot([final[pop][ilvl] for pop in final.keys()],
                                        axis=("x", "Vx"),
                                        bins=(xbins, vbins))
                else:
                    fig, ax = dist_plot([final[pop][ilvl] for pop in final.keys()],
                                        axis=("x", "Vx"),bins=(xbins, vbins), ax=ax)
            else:
                if ilvl==0:
                    fig, ax = dist_plot([final[pop][ilvl] for pop in final.keys()],
                                        axis=axis,
                                        bins=(vbins, vbins))
                else:
                    fig, ax = dist_plot([final[pop][ilvl] for pop in final.keys()],
                                        axis=axis,bins=(vbins, vbins), ax=ax)
                
        
    else:
        cpt = 0
        for lvl_nbr,level in self.levels(time).items():
            if lvl_nbr not in usr_lvls:
                continue
            for ip, patch in enumerate(level.patches):
                if len(pops)==0:
                    pops = list(patch.patch_datas.keys())

                for pop in pops:
                    tmp = copy.copy(patch.patch_datas[pop].dataset)

                    # select particles
                    if "select" in kwargs:
                        selected = kwargs["select"](tmp)
                    else:
                        selected = tmp
                    if cpt == 0:
                        final = selected
                    else:
                        final.add(selected)

                    cpt += 1
        return final, dist_plot(final, **kwargs)
    

In [102]:
r2 = Run("/home/aunai/phare_jobs/tests/phase_space/run002/")
ions = r2.GetParticles(0., ["protons", "beam"])
protons = r2.GetParticles(0., ["protons"])
beam = r2.GetParticles(0., ["beam"])
#particles = finest_part_data(ions)

In [103]:
hdist_plot(ions, finest=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [6]:
41*0.2

8.200000000000001

In [10]:
71*0.1

7.1000000000000005

In [9]:
int(70/2)*0.2+0.05

7.05

In [5]:
fig, ax = plt.subplots(figsize=(10,5))
dv = 0.05
vmin = -2
vmax = 2
vbins = np.arange(vmin, vmax+dv, dv)
fig, ax = dist_plot([particles[pop][0] for pop in particles.keys()],
                     axis=("x", "Vx"),bins=(np.arange(0,20.2,0.2), vbins), ax=ax)

ax.axvline(20*0.2, ls="-", color="w")
ax.axvline(41*0.2, ls="-", color="w")
ax.axvline(25*0.2, ls="-", color="r")
ax.axvline(71*0.1+0.05, ls="-", color="r")

fig,ax = dist_plot([particles[pop][1] for pop in particles.keys()],
                   axis=("x", "Vx"),
                   bins=(np.arange(4,8.21,0.1),vbins), ax=ax)

fig,ax, = dist_plot([particles[pop][2] for pop in particles.keys()],
                    axis=("x", "Vx"), bins=(np.arange(5,7.16,0.05),vbins), ax=ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [7]:
fig, ax = plt.subplots(figsize=(10,5))
dv = 0.05
vmin = -2
vmax = 2
vbins = np.arange(vmin, vmax+dv, dv)
fig, ax = dist_plot([particles[pop][0] for pop in particles.keys()],
                     axis=("x", "Vx"),bins=(np.arange(0,20.2,0.2), vbins), ax=ax)

ax.axvline(20*0.2, ls="-", color="w")
ax.axvline(41*0.2, ls="-", color="w")
ax.axvline(25*0.2, ls="-", color="r")
ax.axvline(71*0.1+0.05, ls="-", color="r")

fig,ax = dist_plot([particles[pop][1] for pop in particles.keys()],
                   axis=("x", "Vx"),
                   bins=(np.arange(4,8.21,0.1),vbins), ax=ax)

fig,ax, = dist_plot([particles[pop][2] for pop in particles.keys()],
                    axis=("x", "Vx"), bins=(np.arange(5,7.16,0.05),vbins), ax=ax)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
